This file contains the code to find the most suitable K value for the KNN filling algorithm
Since Logistic regression performs and worst among all algorithm, we don't use it here

In [2]:

from dask.distributed import Client
import dask.dataframe as dd
import dask.array as da
import dask_ml
import joblib
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.csvcsv)
import sklearn as sk
# import statsmodels.api as sm
# # Cross Validation Classification Accuracy
# from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold,cross_val_score
# from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing 
# from sklearn.impute import KNNImputer
# #import matplotlib.pyplot as plot
# from matplotlib import pyplot
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score,f1_score
from sklearn import tree
# import statistics
import time
import warnings
warnings.filterwarnings('ignore')
# plotting errorbar graph for visual representation of the performance of the classifiers
# import matplotlib.pyplot as plt
# from matplotlib.transforms import Affine2D
import random
from xgboost import XGBClassifier
# import shutil
# import os
# from our_evaluate_sepsis_score import evaluate_performance
import our_functions_library as flib

In [3]:
# only run this once
client = Client()

In [4]:
client

Client Scheduler: tcp://127.0.0.1:54049 Dashboard: http://127.0.0.1:54048/status,Cluster Workers: 4 Cores: 16 Memory: 16.95 GB


In [24]:
# Only run this when the client has started
client.shutdown()

In [25]:
client.close()

In [95]:
# dataType={'Age': 'int', 'Gender': bool, 'Patient_id': int, 'time': 'int8', 'SepsisLabel':'float16'}
filename = '../../raw_data/raw_data_400.csv' # use raw dataset
originalData = dd.read_csv(filename) # read csv data into Dask DataFrame
Uniq_ID= np.unique(originalData['Patient_id'])
print("Data size:",originalData.shape)
print('Patient id size:',len(Uniq_ID))
print(Uniq_ID)

X_columns = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2','BaseExcess', 'HCO3', 'FiO2', 'pH', 
             'PaCO2', 'SaO2', 'AST', 'BUN','Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
             'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium','Bilirubin_total', 'TroponinI', 'Hct', 
             'Hgb', 'PTT', 'WBC','Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2','HospAdmTime',
             'ICULOS', 'Patient_id', 'time']
y_columns = ['Patient_id', 'SepsisLabel']

# To test if these 3 columns is useful
# originalData = originalData.drop(['Unit1', 'Unit2', 'EtCO2'], axis=1)
# X_columns.remove('Unit1')
# X_columns.remove( 'Unit2')
# X_columns.remove( 'EtCO2')

# Initialize the empty array X_train, X_test, y_train, y_test
# X_train = dd.from_pandas(pd.DataFrame(columns = X_columns),npartitions=2)
# X_test = dd.from_pandas(pd.DataFrame(columns = X_columns),npartitions=2)
# y_train = dd.from_pandas(pd.DataFrame(columns = y_columns),npartitions=2)
# y_test = dd.from_pandas(pd.DataFrame(columns = y_columns),npartitions=2)

# Below are the lists for KNN results of different K value
KNN_UtilityScore_mean = []
KNN_UtilityScore_std = []
KNN_F1Score_mean = []
KNN_F1Score_std = []
KNN_auroc_mean = []
KNN_auprc_mean = []
KNN_accuracy_mean = []
KNN_accuracy_std = []
KNN_positiveprediction_mean = []
KNN_baseline_mean = [ ]
KNN_total_Time= [ ]
fillmethod =""  

Data size: (Delayed('int-c015dd4e-8a42-47a4-bccd-366966d4920f'), 43)
Patient id size: 400
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 2

In [96]:
originalData.compute().shape

(15348, 43)

In [97]:
#This block will shuffle the id sets of patients in a fix manner, so for every time you run the code, the dataset and trainset are always the same.
seed = 2
random.seed(seed)
if(np.array_equal(range(1,21),Uniq_ID[:20]) ):
    print("The first 20 patient ids are the same. Uniq id hasn't been shuffled, and it will be shuffeld now.")   
    random.shuffle(Uniq_ID)# randomly sorted the patient IDs 
#     Uniq_ID = da.from_array(Uniq_ID)
else:
    print("They are not the same. Uniq_id has already been shuffled.")    
print("The original uniq id set is:\n",Uniq_ID[0:20])
print("The fixed shuffelld id set, it should be 211 275 153 189 184 110 124  49...\n",Uniq_ID[:20])

The first 20 patient ids are the same. Uniq id hasn't been shuffled, and it will be shuffeld now.
The original uniq id set is:
 [211 275 153 189 184 110 124  49 345  73 284  20  71 328 136 241 171 141
 329 204]
The fixed shuffelld id set, it should be 211 275 153 189 184 110 124  49...
 [211 275 153 189 184 110 124  49 345  73 284  20  71 328 136 241 171 141
 329 204]


In [98]:
def KNN_reset():
    KNN_UtilityScore_mean.clear()
    KNN_UtilityScore_std.clear()
    KNN_F1Score_mean.clear()
    KNN_F1Score_std.clear()
    KNN_auroc_mean.clear()
    KNN_auprc_mean.clear()
    KNN_accuracy_mean.clear()
    KNN_accuracy_std.clear()
    KNN_positiveprediction_mean.clear()
    KNN_baseline_mean.clear()
    KNN_total_Time.clear()
    
# def generateTrainDataSet(test_patienIds, X_train, y_train):
#     train_data= originalData[~(originalData.Patient_id.isin(test_patienIds))]
#     X_train = train_data[X_columns]
#     y_train = train_data[y_columns]  
#     return X_train, y_train
 
# def generateTestDataSet(patienIds, X_test, y_test):
#     print("test_patienIds: \n", patienIds)
#     test_data = originalData[originalData['Patient_id'].isin(patienIds)]
#     X_test = test_data[X_columns]
#     y_test = test_data[y_columns]
#     return X_test, y_test

def generateTrainDataSet(test_patienIds):
    train_data= originalData[~(originalData.Patient_id.isin(test_patienIds))]
    X_train = train_data[X_columns]
    y_train = train_data[y_columns]  
    return X_train, y_train
 
def generateTestDataSet(patienIds):
    print("test_patienIds: \n", patienIds)
    test_data = originalData[originalData['Patient_id'].isin(patienIds)]
    X_test = test_data[X_columns]
    y_test = test_data[y_columns]
    return X_test, y_test

In [99]:
model=sk.tree.DecisionTreeClassifier(random_state=2)
KforKFold=10
KforKNN=5
fillmethod=""
# Code for K-fold algorithm 
# def KFold_patient(model, KforKFold=10,KforKNN=5, fillmethod=""):
global X_train, X_test, y_train, y_test
print("Datasize",len(Uniq_ID) ,"with K Fold of ",KforKFold ," folds filling by KNN =",KforKNN)

# initialisation of the array for storing the different intermediate results
auroc_model = []
auprc_model = []
physio_accuracy_model = []
f_measure_model = []
utility_score_model = []
positivepredictions = []
baseline_model= []
#The unique id sets have been created and shuffled in a fix manner in the third block, you can just use it here and no more any other manipulation
idSets = np.array_split(Uniq_ID, KforKFold)# divide the ids into K groups
#This for loop is for Kfold, calculating the results for K times

start = time.time()# time indicator for how long the Kfold func takes

#write the idset data into a file
open("test.txt", "w").close() # clear contents of existing file
for i in range(KforKFold):
    splitted = " ".join( repr(e) for e in idSets[i])
    file1 = open("test.txt","a")
    file1.write("\n\n")
    file1.write(str("[" +splitted+"]"))
    file1.write("\n\n")
    file1.close()
print("Time for writing id to file:", round(time.time()- start,3))

# for i in range(KforKFold):
for i in range(1):
    print("for the",i+1,"th iteration: ")#,idSets[i])   
    t1 = time.time()
    X_test,y_test = generateTestDataSet(idSets[i])
    print("Time for splitting id to test dataset:", round(time.time()- t1,3))
    t1 = time.time()
    X_train, y_train = generateTrainDataSet(idSets[i])
    print("Time for splitting id to train dataset:", round(time.time()- t1,3))

    #Now the train and test dataset is generated 
    patientID_ytest = y_test['Patient_id']
    y_train = y_train.drop('Patient_id', 1)
    y_test = y_test.drop('Patient_id', 1) 
    X_train=X_train.astype('float32')
    X_test=X_test.astype('float32') 
#     y_train=y_train.astype('i1')
#     y_test=y_test.astype('i1')
    y_train=y_train.astype('bool')
    y_test=y_test.astype('bool')
    print(X_train)
    
    #fill the missing data
    if X_train.isnull().values.any() or X_test.isnull().values.any() :
        t1= time.time()
        print("X_train or X_test contains NaN values, KNN is performed.")
#         X_train, X_test,fillmethod = flib.MeanFilling(X_train,X_test, fillmethod)  
        X_train, X_test, fillmethod = flib.KNNfilling(X_train, X_test, KforKNN, fillmethod)
#         imputer = sk.impute.KNNImputer(n_neighbors = KforKNN)
#         with joblib.parallel_backend("dask",scatter=[X_train, y_train]):
#             imputer.fit(X_train)
#             x_train=imputer.transform(X_train)
#             x_test=imputer.transform(X_test)
        print("The type of X_train after KNN filling is",type(X_train))
        print("Time for NaN values filling: ", round(time.time()- t1,2) )
        X_train = dd.from_array(X_train)
        X_test = dd.from_array(X_test)
#         if np.isnan(X_train).any() or np.isnan(X_test).any() :
#             print("X_train_impute or X_test_impute still contains NaN values")        
    else:
        print("X_train or X_test have all been filled: ") 
    
    print("The type of X_train after KNN filling is",type(X_train))
    
    #Scale the data for logistic regression
    if ( type(model).__name__ == 'LogisticRegression'):
        t1 = time.time()
        print("scaled the data!")
        scaler = dask_ml.preprocessing.StandardScaler()
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        print("Time for standard scaling: ", round(time.time()- t1,2) )

               
    #fit the model and predict
    with joblib.parallel_backend("dask",scatter=[X_train, y_train]):
        model.fit(X_train, y_train)
        y_predicted = model.predict(X_test).astype(int)   
        y_predicted_probobility = model.predict_proba(X_test) 
#     y_labels = y_test.astype(int).compute().to_numpy()
    y_labels = y_test['SepsisLabel'].astype(int).reset_index(drop=True)
    
    del X_train
    del X_test

    print("columns of y_predicted_probobility is: ",len(y_predicted_probobility[0]))
    if(len(y_predicted_probobility[0]) ==2 ):
        y_predicted_probobility =  y_predicted_probobility[:,1].round(4)
    elif(len(y_predicted_probobility[0]) == 1):     
        y_predicted_probobility =  y_predicted_probobility.round(4)
    
    auroc, auprc, physio_accuracy, f_measure, utility_score = evaluate_performance_dask(y_labels.compute(), y_predicted, y_predicted_probobility,patientID_ytest.compute()) 
    
    auroc_model.append(round(auroc,4))
    auprc_model.append(round(auprc,4))
    physio_accuracy_model.append(round(physio_accuracy,4))
    f_measure_model.append(round(f_measure,4))
    utility_score_model.append(round(utility_score,4))
    positivepredictions.append(np.sum(y_predicted))        
    baseline_model.append(round( (1 - y_test.mean().compute() )*100 , 2 ))
    print("\ny_test size:",y_test.shape, '1´s in y_test',y_test.sum())
    print("The number of 1 (SepsisLabel) in this prediction: ", np.sum(y_predicted))
    print("Time spent in this KFold iteration",round(time.time()-t1,2),"sec.\n")
    print("******************************************************************")    
print("\nEvaluation parameters of the utiltiy evaluation function:")
print('auroc of model:', auroc_model)
print('auprc of model:', auprc_model)
print('utility F1 of model:', f_measure_model)
print('Utility accuracy of model:', physio_accuracy_model) 
print('Utility score of model:', utility_score_model)
print("Positive Prediction: ", positivepredictions)
print('Baseline model:', baseline_model)  
KNN_auroc_mean.append(np.mean(auroc_model))
KNN_auprc_mean.append(np.mean(auprc_model))
KNN_F1Score_mean.append(np.mean(f_measure_model))
KNN_F1Score_std.append(np.std(f_measure_model))
KNN_accuracy_mean.append(np.mean(physio_accuracy_model))
KNN_accuracy_std.append(np.std(physio_accuracy_model))
KNN_UtilityScore_mean.append(np.mean(utility_score_model))
KNN_UtilityScore_std.append(np.std(utility_score_model))
KNN_positiveprediction_mean.append(np.mean(positivepredictions))
KNN_baseline_mean.append(np.mean(baseline_model))

totalTime=round(time.time()- start,2)
KNN_total_Time.append(totalTime)
print("\nTotal Time spent in  KFold function",totalTime,"sec.\n")

Datasize 400 with K Fold of  10  folds filling by KNN = 5
Time for writing id to file: 0.002
for the 1 th iteration: 
test_patienIds: 
 [211 275 153 189 184 110 124  49 345  73 284  20  71 328 136 241 171 141
 329 204 157 290 305 281 323  57 208  41 366 220 120 118 244  22 325 222
 178 369 381  52]
Time for splitting id to test dataset: 0.003
Time for splitting id to train dataset: 0.003
Dask DataFrame Structure:
                    HR    O2Sat     Temp      SBP      MAP      DBP     Resp    EtCO2 BaseExcess     HCO3     FiO2       pH    PaCO2     SaO2      AST      BUN Alkalinephos  Calcium Chloride Creatinine Bilirubin_direct  Glucose  Lactate Magnesium Phosphate Potassium Bilirubin_total TroponinI      Hct      Hgb      PTT      WBC Fibrinogen Platelets      Age   Gender    Unit1    Unit2 HospAdmTime   ICULOS Patient_id     time
npartitions=1                                                                                                                                               

In [ ]:
# print(type(y_labels.compute()))
# print(y_labels)
# y_labels.compute().rename("labels")
y_test.compute().mean()

In [24]:
from sys import getsizeof
print(getsizeof(X_train))
print(getsizeof(X_test))
print(X_train.shape)
print(X_test.shape)
# X_test.isnull().values.any()

112
112
(13813, 42)
(1535, 42)


In [73]:
# print("X_train size is: ",X_train.size * X_train.itemsize)
# print("X_test size is: ",X_test.size * X_test.itemsize)
print("X_train size is: ",X_train.memory_usage(deep=True).sum().compute())
print("X_test size is: ",X_test.memory_usage(deep=True).sum().compute())
print("y_train size is: ",y_train.memory_usage(deep=True).sum().compute())
print("y_test size is: ",y_test.memory_usage(deep=True).sum().compute())

X_train size is:  2320712
X_test size is:  258008
y_train size is:  124317
y_test size is:  13815


In [ ]:
np.sum(y_labels)
print(y_labels)
print(y_predicted)
print(type(y_predicted))
print(y_predicted_probobility)
print(type(y_predicted_probobility))

In [100]:
# X_train.loc[0:0].compute()
X_train[0:0].compute()

NameError: name 'X_train' is not defined

In [64]:
def Print_DATA():
    print("Numbers of K:",len(KNN_accuracy_mean))
    print("KNN_UtilityScore_mean", KNN_UtilityScore_mean)
    print("KNN_UtilityScore_std", KNN_UtilityScore_std)
    print("KNN_F1Score_mean", KNN_F1Score_mean)
    print("KNN_F1Score_std", KNN_F1Score_std)
    print("KNN_auroc_mean", KNN_auroc_mean)
    print("KNN_auprc_mean", KNN_auprc_mean)
    print("KNN_accuracy_mean", KNN_accuracy_mean)
    print("KNN_accuracy_std", KNN_accuracy_std)
    print("KNN_positiveprediction_mean", KNN_positiveprediction_mean)
    print("KNN_baseline_mean", KNN_baseline_mean)
    print("KNN_total_time", KNN_total_Time)      

# def findBestKforKNN(model, KforKFold=10,KforKNNstart=1,KforKNNend=10, stepsize=5):
#     if(KforKNNend<=1):
#         print("K must be a interger larger than 1")
#         return
#     KNN_reset()
#     step=stepsize
#     for i in range(KforKNNstart,KforKNNend+1,step) : 
#         print("KNN of K = ",i)    
#         KFold_patient(model,10,i)

In [ ]:
#logisticRegressionModel = LogisticRegression(C=0.1,random_state=2,solver='saga', max_iter=1000,penalty='l1')
logisticRegressionModel = LogisticRegression(C=0.1,random_state=2,solver='lbfgs', max_iter=1000)
decisionTreeModel = tree.DecisionTreeClassifier(random_state=2)
randomForestModel = RandomForestClassifier(random_state=2)
XGBoostModel = XGBClassifier(random_state=2, n_jobs=4)
#AdaBoostModel = AdaBoostClassifier(random_state=2)

In [ ]:
#fill the missing data with one function
print(filename)
#print(originalData)
if (originalData.isnull().values.any()):
    print('Data isn´t all filled before K-Fold Func')
#data,fillmethod = flib.linearFillingAll(originalData,1, True)#forwardfilling + fill other missing data with 0

In [ ]:
# KFold_patient(decisionTreeModel,10,5)
Print_DATA()

In [65]:
def evaluate_performance_dask(labels,predictions,probabilities,patientids):
    # print("****************************************************************************")
    # print(type(labels))
    # print(labels)
    # print("*****************************")
    # print(type(predictions))
    # print(predictions)
    # print("*****************************")
    # print(type(patientids))
    # print(patientids)
    # print("*****************************")
    # print(type(probabilities))
    # print(probabilities)
    # print("*****************************")
    if ( not( len(labels) == len(predictions) == len(probabilities) )):
        print("The predicted data is not the same in size! Evaluation function will stop here!" )
        return
    # Set parameters.
    label_header       = 'SepsisLabel'
    prediction_header  = 'PredictedLabel'
    probability_header = 'PredictedProbability'
    dt_early   = -12
    dt_optimal = -6
    dt_late    = 3
    max_u_tp = 1
    min_u_fn = -2
    u_fp     = -0.05
    u_tn     = 0
    
    auroc, auprc        = compute_auc(labels, probabilities)
    accuracy, f_measure = compute_accuracy_f_measure(labels, predictions)
    
    patientids = patientids.astype('int')
    patientdata = patientids.reset_index()
#     print(patientids)
#     patientdata = patientdata.join(pd.Series(labels.flatten(),name='labels'))
    patientdata = patientdata.join(labels.rename('labels'))
    patientdata = patientdata.join(pd.Series(predictions.flatten(),name='predictions'))
    patientdata = patientdata.drop("index",1)

    # print("After the joining:")
    # print(patientdata.columns)
    # print(patientdata)

    uniq_ids = np.unique(patientdata['Patient_id']) 
    num_patients = len(uniq_ids)
    dataByPatient = patientdata.groupby('Patient_id')

    #print(uniq_ids)
    
    # Compute utility.
    observed_utilities = np.zeros(num_patients)
    best_utilities     = np.zeros(num_patients)
    worst_utilities    = np.zeros(num_patients)
    inaction_utilities = np.zeros(num_patients)

    for i in range(num_patients):
        k = uniq_ids[i]
        # print(" for the patient of id:",k)
        # print(dataByPatient.get_group(k))

        labels = dataByPatient.get_group(k).loc[:, 'labels'].to_numpy()
        num_rows          = len(labels)
        observed_predictions = dataByPatient.get_group(k).loc[:, 'predictions'].to_numpy()

        #print("labels",labels)
        #print("observered_prediction",observed_predictions)

        best_predictions     = np.zeros(num_rows)
        worst_predictions    = np.zeros(num_rows)
        inaction_predictions = np.zeros(num_rows)

        if np.any(labels):
            t_sepsis = np.argmax(labels) - dt_optimal
            best_predictions[max(0, t_sepsis + dt_early) : min(t_sepsis + dt_late + 1, num_rows)] = 1
        worst_predictions = 1 - best_predictions

        observed_utilities[i] = compute_prediction_utility(labels, observed_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        best_utilities[i]     = compute_prediction_utility(labels, best_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        worst_utilities[i]    = compute_prediction_utility(labels, worst_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        inaction_utilities[i] = compute_prediction_utility(labels, inaction_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)

    unnormalized_observed_utility = np.sum(observed_utilities)
    unnormalized_best_utility     = np.sum(best_utilities)
    unnormalized_worst_utility    = np.sum(worst_utilities)
    unnormalized_inaction_utility = np.sum(inaction_utilities)

    normalized_observed_utility = (unnormalized_observed_utility - unnormalized_inaction_utility) / (unnormalized_best_utility - unnormalized_inaction_utility)

    return auroc, auprc, accuracy, f_measure, normalized_observed_utility


# The load_column function loads a column from a table.
#
# Inputs:
#   'filename' is a string containing a filename.
#
#   'header' is a string containing a header.
#
# Outputs:
#   'column' is a vector containing a column from the file with the given
#   header.
#
# Example:
#   Omitted.

def load_column(filename: object, header: object, delimiter: object) -> object:
    column = []
    with open(filename, 'r') as f:
        for i, l in enumerate(f):
            arrs = l.strip().split(delimiter)
            if i == 0:
                try:
                    j = arrs.index(header)
                except:
                    raise Exception('{} must contain column with header {} containing numerical entries.'.format(filename, header))
            else:
                if len(arrs[j]):
                    column.append(float(arrs[j]))
    return np.array(column)

""
# The compute_auc function computes AUROC and AUPRC as well as other summary
# statistics (TP, FP, FN, TN, TPR, TNR, PPV, NPV, etc.) that can be exposed
# from this function.
#
# Inputs:
#   'labels' is a binary vector, where labels[i] == 0 if the patient is not
#   labeled as septic at time i and labels[i] == 1 if the patient is labeled as
#   septic at time i.
#
#   'predictions' is a probability vector, where predictions[i] gives the
#   predicted probability that the patient is septic at time i.  Note that there
#   must be a prediction for every label, i.e, len(labels) ==
#   len(predictions).
#
# Outputs:
#   'auroc' is a scalar that gives the AUROC of the algorithm using its
#   predicted probabilities, where specificity is interpolated for intermediate
#   sensitivity values.
#
#   'auprc' is a scalar that gives the AUPRC of the algorithm using its
#   predicted probabilities, where precision is a piecewise constant function of
#   recall.
#
# Example:
#   In [1]: labels = [0, 0, 0, 0, 1, 1]
#   In [2]: predictions = [0.3, 0.4, 0.6, 0.7, 0.8, 0.8]
#   In [3]: auroc, auprc = compute_auc(labels, predictions)
#   In [4]: auroc
#   Out[4]: 1.0
#   In [5]: auprc
#   Out[5]: 1.0

def compute_auc(labels, predictions, check_errors=True):
    # Check inputs for errors.
    #print('labels inside auc_fucn',labels)
    if check_errors:
        if len(predictions) != len(labels):
            raise Exception('Numbers of predictions and labels must be the same.')

        for label in labels:
            if not label in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

        for prediction in predictions:
            if not 0 <= prediction <= 1:
                warnings.warn('Predictions do not satisfy 0 <= prediction <= 1.')

    # Find prediction thresholds.
    thresholds = np.unique(predictions)[::-1]
    if thresholds[0] != 1:
        thresholds = np.insert(thresholds, 0, 1)
    if thresholds[-1] == 0:
        thresholds = thresholds[:-1]

    n = len(labels)
    m = len(thresholds)

    # Populate contingency table across prediction thresholds.
    tp = np.zeros(m)
    fp = np.zeros(m)
    fn = np.zeros(m)
    tn = np.zeros(m)

    # Find indices that sort the predicted probabilities from largest to
    # smallest.
    idx = np.argsort(predictions)[::-1]

    i = 0
    for j in range(m):
        # Initialize contingency table for j-th prediction threshold.
        if j == 0:
            tp[j] = 0
            fp[j] = 0
            fn[j] = np.sum(labels)
            tn[j] = n - fn[j]
        else:
            tp[j] = tp[j - 1]
            fp[j] = fp[j - 1]
            fn[j] = fn[j - 1]
            tn[j] = tn[j - 1]

        # Update contingency table for i-th largest predicted probability.
        while i < n and predictions[idx[i]] >= thresholds[j]:
            if labels[idx[i]]:
                tp[j] += 1
                fn[j] -= 1
            else:
                fp[j] += 1
                tn[j] -= 1
            i += 1

    # Summarize contingency table.
    tpr = np.zeros(m)
    tnr = np.zeros(m)
    ppv = np.zeros(m)
    npv = np.zeros(m)

    for j in range(m):
        if tp[j] + fn[j]:
            tpr[j] = tp[j] / (tp[j] + fn[j])
        else:
            tpr[j] = 1
        if fp[j] + tn[j]:
            tnr[j] = tn[j] / (fp[j] + tn[j])
        else:
            tnr[j] = 1
        if tp[j] + fp[j]:
            ppv[j] = tp[j] / (tp[j] + fp[j])
        else:
            ppv[j] = 1
        if fn[j] + tn[j]:
            npv[j] = tn[j] / (fn[j] + tn[j])
        else:
            npv[j] = 1

    # Compute AUROC as the area under a piecewise linear function with TPR /
    # sensitivity (x-axis) and TNR / specificity (y-axis) and AUPRC as the area
    # under a piecewise constant with TPR / recall (x-axis) and PPV / precision
    # (y-axis).
    auroc = 0
    auprc = 0
    for j in range(m-1):
        auroc += 0.5 * (tpr[j + 1] - tpr[j]) * (tnr[j + 1] + tnr[j])
        auprc += (tpr[j + 1] - tpr[j]) * ppv[j + 1]

    return auroc, auprc

# The compute_accuracy_f_measure function computes the accuracy and F-measure
# for a patient.
#
# Inputs:
#   'labels' is a binary vector, where labels[i] == 0 if the patient is not
#   labeled as septic at time i and labels[i] == 1 if the patient is labeled as
#   septic at time i.
#
#   'predictions' is a binary vector, where predictions[i] == 0 if the patient
#   is not predicted to be septic at time i and predictions[i] == 1 if the
#   patient is predicted to be septic at time i.  Note that there must be a
#   prediction for every label, i.e, len(labels) == len(predictions).
#
# Output:
#   'accuracy' is a scalar that gives the accuracy of the predictions using its
#   binarized predictions.
#
#   'f_measure' is a scalar that gives the F-measure of the predictions using its
#   binarized predictions.
#
# Example:
#   In [1]: labels = [0, 0, 0, 0, 1, 1]
#   In [2]: predictions = [0, 0, 1, 1, 1, 1]
#   In [3]: accuracy, f_measure = compute_accuracy_f_measure(labels, predictions)
#   In [4]: accuracy
#   Out[4]: 0.666666666667
#   In [5]: f_measure
#   Out[5]: 0.666666666667

def compute_accuracy_f_measure(labels, predictions, check_errors=True):
    # Check inputs for errors.
    if check_errors:
        if len(predictions) != len(labels):
            raise Exception('Numbers of predictions and labels must be the same.')

        for label in labels:
            if not label in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

        for prediction in predictions:
            if not prediction in (0, 1):
                raise Exception('Predictions must satisfy prediction == 0 or prediction == 1.')

    # Populate contingency table.
    n = len(labels)
    tp = 0
    fp = 0
    fn = 0
    tn = 0

    for i in range(n):
        if labels[i] and predictions[i]:
            tp += 1
        elif not labels[i] and predictions[i]:
            fp += 1
        elif labels[i] and not predictions[i]:
            fn += 1
        elif not labels[i] and not predictions[i]:
            tn += 1

    # Summarize contingency table.
    if tp + fp + fn + tn:
        accuracy = float(tp + tn) / float(tp + fp + fn + tn)
    else:
        accuracy = 1.0

    if 2 * tp + fp + fn:
        f_measure = float(2 * tp) / float(2 * tp + fp + fn)
    else:
        f_measure = 1.0

    return accuracy, f_measure

# The compute_prediction_utility function computes the total time-dependent
# utility for a patient.
#
# Inputs:
#   'labels' is a binary vector, where labels[i] == 0 if the patient is not
#   labeled as septic at time i and labels[i] == 1 if the patient is labeled as
#   septic at time i.
#
#   'predictions' is a binary vector, where predictions[i] == 0 if the patient
#   is not predicted to be septic at time i and predictions[i] == 1 if the
#   patient is predicted to be septic at time i.  Note that there must be a
#   prediction for every label, i.e, len(labels) == len(predictions).
#
# Output:
#   'utility' is a scalar that gives the total time-dependent utility of the
#   algorithm using its binarized predictions.
#
# Example:
#   In [1]: labels = [0, 0, 0, 0, 1, 1]
#   In [2]: predictions = [0, 0, 1, 1, 1, 1]
#   In [3]: utility = compute_prediction_utility(labels, predictions)
#   In [4]: utility
#   Out[4]: 3.388888888888889

def compute_prediction_utility(labels, predictions, dt_early=-12, dt_optimal=-6, dt_late=3.0, max_u_tp=1, min_u_fn=-2, u_fp=-0.05, u_tn=0, check_errors=True):
    # Check inputs for errors.
    if check_errors:
        if len(predictions) != len(labels):
            raise Exception('Numbers of predictions and labels must be the same.')

        for label in labels:
            if not label in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

        for prediction in predictions:
            if not prediction in (0, 1):
                raise Exception('Predictions must satisfy prediction == 0 or prediction == 1.')

        if dt_early >= dt_optimal:
            raise Exception('The earliest beneficial time for predictions must be before the optimal time.')

        if dt_optimal >= dt_late:
            raise Exception('The optimal time for predictions must be before the latest beneficial time.')

    # Does the patient eventually have sepsis?
    if np.any(labels):
        is_septic = True
        t_sepsis = np.argmax(labels) - dt_optimal
    else:
        is_septic = False
        t_sepsis = float('inf')

    n = len(labels)

    # Define slopes and intercept points for utility functions of the form
    # u = m * t + b.
    m_1 = float(max_u_tp) / float(dt_optimal - dt_early)
    b_1 = -m_1 * dt_early
    m_2 = float(-max_u_tp) / float(dt_late - dt_optimal)
    b_2 = -m_2 * dt_late
    m_3 = float(min_u_fn) / float(dt_late - dt_optimal)
    b_3 = -m_3 * dt_optimal

    # Compare predicted and true conditions.
    u = np.zeros(n)
    for t in range(n):
        if t <= t_sepsis + dt_late:
            # TP
            if is_septic and predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = max(m_1 * (t - t_sepsis) + b_1, u_fp)
                elif t <= t_sepsis + dt_late:
                    u[t] = m_2 * (t - t_sepsis) + b_2
            # FP
            elif not is_septic and predictions[t]:
                u[t] = u_fp
            # FN
            elif is_septic and not predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = 0
                elif t <= t_sepsis + dt_late:
                    u[t] = m_3 * (t - t_sepsis) + b_3
            # TN
            elif not is_septic and not predictions[t]:
                u[t] = u_tn

    # Find total utility for patient.
    return np.sum(u)